In [25]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import glob

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'USDC-WETH'

In [26]:
lp_action = pd.DataFrame(columns = ['POOL_NAME', 'EVENT_NAME', 'BLOCK_NUMBER', 'BLOCK_TIMESTAMP', 'SENDER', 'NF_TOKEN_ID', 'AMOUNT0_ADJUSTED', 'TOKEN0_SYMBOL', 'AMOUNT0_USD',
                                    'AMOUNT1_ADJUSTED', 'TOKEN1_SYMBOL', 'AMOUNT1_USD', 'TICK_LOWER', 'TICK_UPPER', 'PRICE_LOWER', 'PRICE_UPPER', 'L'])

for file_name in [file for file in glob.glob(os.path.join(data_dir, "uniswapv3_lp_action_LP") + '/*.csv') if pool_name in file]:

    lp_action = pd.concat([lp_action, pd.read_csv(file_name)])

# Final cleaning
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime(lp_action['BLOCK_TIMESTAMP'])
lp_action['AMOUNT0_ADJUSTED'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT0_ADJUSTED'], lp_action['AMOUNT0_ADJUSTED'])
lp_action['AMOUNT0_USD'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT0_USD'], lp_action['AMOUNT0_USD'])
lp_action['AMOUNT1_ADJUSTED'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT1_ADJUSTED'], lp_action['AMOUNT1_ADJUSTED'])
lp_action['AMOUNT1_USD'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['AMOUNT1_USD'], lp_action['AMOUNT1_USD'])
lp_action['L'] = np.where(lp_action['EVENT_NAME'] == 'Decrease Liquidity', -lp_action['L'], lp_action['L'])

# Round Liquidity Parameter to 5 decimal point
lp_action['L'] = round(lp_action['L'], 5)

lp_action.reset_index(drop = True, inplace = True)
lp_action.head()

,POOL_NAME,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER,L
0,USDC-WETH 3000 60,Increase Liquidity,12370624,2021-05-04 23:10:00,0x3b8ccaa89fcd432f1334d35b10ff8547001ce3e5,37,0.000000,USDC,0.000000,0.010000,WETH,33.542413,192180,193380,3999.753253,4509.682144,10.86051
1,USDC-WETH 3000 60,Increase Liquidity,12370671,2021-05-04 23:19:42,0xe14722b5f1aa6ea161e547c5ca598068ec1b3bc0,38,50000.000000,USDC,49997.486569,9.205484,WETH,30877.415059,192660,199800,2104.893286,4298.340735,4303.36967
2,USDC-WETH 3000 60,Increase Liquidity,12371434,2021-05-05 02:10:21,0x5eefc9306f11a824762ccdaedac41049efc7fcc8,41,167.505955,USDC,167.501413,0.044750,WETH,150.584382,195060,195540,3222.777198,3381.235151,229.98572
3,USDC-WETH 3000 60,Increase Liquidity,12371483,2021-05-05 02:21:03,0x5eefc9306f11a824762ccdaedac41049efc7fcc8,41,0.010000,USDC,0.010000,0.000003,WETH,0.008990,195060,195540,3222.777198,3381.235151,0.01373
4,USDC-WETH 3000 60,Decrease Liquidity,12371521,2021-05-05 02:30:25,0x5eefc9306f11a824762ccdaedac41049efc7fcc8,41,-167.515954,USDC,-167.511412,-0.044752,WETH,-150.593372,195060,195540,3222.777198,3381.235151,-229.99945


# Identifying liquidity reallocation 

Since liquidity allocation depends on risk-aversion of the liquidity provider, we want to identify events of liquidity reallocation which can signal when a liquidity provider changes their risk aversion.

### Across multiple fee pools

In [28]:
multi_pool_LPs = lp_action.groupby(['SENDER'])['POOL_NAME'].nunique().reset_index()
multi_pool_LPs = multi_pool_LPs[multi_pool_LPs['POOL_NAME'] >= 2]['SENDER']

lp_action = lp_action[lp_action['SENDER'].isin(multi_pool_LPs)]

multi_pool_LPs.head()

0    0x00000000002108f4085d2fd429669ce8f9851241
1    0x000000000088e0120f9e6652cc058aec07564f69
2    0x0000c3caa36e2d9a8cd5269c976ede05018f0000
3    0x00080000158a9930aeb508823103b0a977161c47
4    0x0008d343091ef8bd3efa730f6aae5a26a285c7a2
Name: SENDER, dtype: object